In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [20]:
train.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,1

In [24]:
train[train['hospital_death']==0]['encounter_id'].count()

83788

In [25]:
train[train['hospital_death']==1]['encounter_id'].count()

7915

In [26]:
83788/ (83788 + 7915)

0.9136887560930395

In [27]:
7915/ (83788 + 7915)

0.08631124390696052

In [21]:
train['hospital_admit_source'].unique()

array(['Floor', 'Emergency Department', 'Operating Room', '',
       'Direct Admit', 'Other Hospital', 'Other ICU', 'ICU to SDU',
       'Recovery Room', 'Chest Pain Center', 'Step-Down Unit (SDU)',
       'Acute Care/Floor', 'PACU', 'ICU', 'Other'], dtype=object)

In [ ]:
#train[train['hospital_admit_source']=='Observation']
train.drop(train.loc[train['hospital_admit_source']=='Observation'].index, inplace=True)

In [ ]:
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)

print(train_categorical_cols)
print(len(train_categorical_cols))
print(len(train_numerical_cols))
print(test_categorical_cols)
print(len(test_categorical_cols))
print(len(test_numerical_cols))

In [ ]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("")
test[test_categorical_cols] = test[test_categorical_cols].fillna("")

In [ ]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()
allcols = train.columns.values.tolist()
inputcols = allcols[4:]
outputcols = allcols[3]

In [ ]:
#inputcols.append('hospital_id')

In [ ]:
X = train[inputcols]
y = train[outputcols]

In [ ]:
X.head()

In [ ]:
train_pool = Pool(data=X,label = y,cat_features=train_categorical_cols,
#                   baseline= X_train[""], ## 
#                   group_id = X_train['hospital_id']
                 )

In [ ]:
### hyperparameter tuning example grid for catboost : 
grid = {'learning_rate': [0.04, 0.05],
        'depth': [10, 11],
        'l2_leaf_reg': [1, 3,9],
        "iterations": [500],
        "custom_metric":['Logloss', 'AUC']}

model = CatBoostClassifier(task_type="CPU")

## can also do randomized search - more efficient typically, especially for large search space - `randomized_search`
grid_search_result = model.grid_search(grid, 
                                       train_pool,
                                       plot=True,
                                       refit = True, #  refit best model on all data
                                      partition_random_seed=42)

print(model.get_best_score())
print("best model params: \n",grid_search_result["params"])

In [ ]:
print('ransom search')

In [28]:
model = CatBoostClassifier(verbose=False, depth = 10, learning_rate=0.04, loss_function='Logloss', boosting_type='Ordered',class_weights=[0.08, 0.91]) #metric_period =50,
model.fit(train_pool, plot=True,silent=True)
print(model.get_best_score())

CatBoostError: catboost/private/libs/options/json_helper.h:153: Can't parse parameter "wait_iterations" with value: true

In [ ]:
'readmission_status' in inputcols

In [ ]:
test_output_y = model.predict(test[inputcols],prediction_type='Probability')[:,1]
test["hospital_death"] = test_output_y
test[["encounter_id","hospital_death"]].to_csv("submission_cat_gs_12.csv",index=False)

In [ ]:
model.get_feature_importance(prettified = True)

In [ ]:
model.get_all_params()

In [ ]:
print('completed')